In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import plotly.graph_objects as go

# Exemples de systèmes dynamiques

## Modèle de Brusselator

La dynamique de la réaction oscillante découverte par Belousov et Zhabotinsky, peut être modélisée par le modèle dit de Brusselator en fonction de deux paramètres $a$ et $b$ :

$$
\left\{\begin{aligned}
{\mathrm d} y_1 & = a - (b+1) y_1 + y_1^2y_2\\
{\mathrm d} y_2 & = b y_1 - y_1^2y_2
\end{aligned}\right.
$$

In [ ]:
class brusselator_model:

    def __init__(self, a, b) :
        self.a = a
        self.b = b
    
    def fcn(self, t, y):
        y1, y2 = y
        a = self.a
        b = self.b 
        y1_dot = a - (b+1)*y1 + y1*y1*y2 
        y2_dot = b*y1 - y1*y1*y2  
        return np.array([y1_dot, y2_dot])

### Solution numérique

In [ ]:
tini = 0. 
tend = 40.
yini = (1.5, 3.)

bm = brusselator_model(a=0.7, b=3)
fcn = bm.fcn  
sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-10, atol=1.e-10)

fig = go.Figure()
fig.add_trace(go.Scatter(x=sol_exa.t, y=sol_exa.y[0], mode="lines", name="y1"))
fig.add_trace(go.Scatter(x=sol_exa.t, y=sol_exa.y[1], mode="lines", name="y2"))

steps = []
for i, a_i in enumerate(np.arange(0.7, 1.71, 0.1)):
    bm = brusselator_model(a=a_i, b=3)
    fcn = bm.fcn  
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-10, atol=1.e-10)
    step = dict(method="update", label = f"{a_i:.2f}", args=[{"x":[sol_exa.t, sol_exa.t],
                                                              "y":[sol_exa.y[0], sol_exa.y[1]]}])
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'a = '}, steps=steps)]

fig.update_layout(sliders=sliders, title="Solution quasi-exacte du système du Brusselator", xaxis_title="t")
fig['layout']['sliders'][0]['pad']=dict(t= 50)
fig.show()

## Système de Lorenz

Le système de Lorenz est un système de trois équations différentielles ordinaires : 

$$
\left\{\begin{aligned}
\dot x & = \sigma \, (y - x)\\
\dot y & = \rho x - y - xz\\ 
\dot z & = xy - \beta z
\end{aligned}\right.
$$

In [ ]:
class lorenz_model():

    def __init__(self, sigma, rho, beta):
        self.sigma = sigma
        self.rho = rho
        self.beta = beta

    def fcn(self, t, xyz):
        x, y, z = xyz
        sigma = self.sigma
        rho = self.rho
        beta = self.beta
        x_dot = sigma*(y-x)
        y_dot = rho*x - y - x*z
        z_dot = x*y - beta*z
        return (x_dot, y_dot, z_dot)

### Solution numérique

In [ ]:
lm = lorenz_model(sigma=10, rho=28, beta=8/3)
fcn = lm.fcn  
    
tini = 0. 
tend = 100.
    
sol_ini = (-10, -7, 35)        
sol = solve_ivp(fcn, (tini, tend), sol_ini, method="RK45", rtol=1.e-8, atol=1.e-10)
x = sol.y[0]; y = sol.y[1]; z = sol.y[2]

fig = go.Figure()
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode="lines"))
fig.update_layout(scene_camera=dict(eye=dict(x=-1.3, y=1.3, z=1.3)), title="Espace des phases")
fig.show()

fig_sol = go.Figure()
fig_sol.add_trace(go.Scatter(x=sol.t, y=x, mode="lines", name="evolution de la variable x"))
fig_sol.update_layout(title="Evolution de la variable x")
fig_sol.show()

### Perturbations des conditions initiales

In [ ]:
sol_ini_pert = (-10.000000001, -7, 35)

sol_pert = solve_ivp(fcn, (tini, tend), sol_ini_pert, method="RK45", rtol=1.e-8, atol=1.e-10)
x_pert = sol_pert.y[0]; y_pert = sol_pert.y[1]; z_pert = sol_pert.y[2]

fig = go.Figure()
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode="lines", name="sol."))
fig.add_trace(go.Scatter3d(x=x_pert, y=y_pert, z=z_pert, mode="lines", name = "sol. perturbée"))
fig.update_layout(scene_camera=dict(eye=dict(x=-1.3, y=1.3, z=1.3)), title="Espace des phases")
fig.show()

fig_sol = go.Figure()
fig_sol.add_trace(go.Scatter(x=sol.t, y=x, mode="lines", name="sol."))
fig_sol.add_trace(go.Scatter(x=sol_pert.t, y=x_pert, mode="lines", name="sol. perturbée"))
fig_sol.update_layout(title="Evolution de la variable x")
fig_sol.show()

## Mécanique céleste : problème à trois corps

In [ ]:
class three_body_model:

    def __init__(self):
        self.m0 = 1.00000597682
        self.m1 = 9.54786104043e-4
        self.m2 = 2.85583733151e-4
        self.G = 2.95912208286e-4

    def fcn(self, t, y):
        q = y[0:9]
        p = y[9:18]
        
        q_dot = np.zeros(9)
        p_dot = np.zeros(9)
        
        m0 = self.m0
        m1 = self.m1
        m2 = self.m2
        G  = self.G
        
        q_dot[0:3] = p[0:3]/m0
        q_dot[3:6] = p[3:6]/m1
        q_dot[6:9] = p[6:9]/m2
        
        p_dot[0:3] = -( G*m0*m1*((q[0:3]-q[3:6])/np.power(np.linalg.norm(q[0:3]-q[3:6]),3)) 
                       +G*m0*m2*((q[0:3]-q[6:9])/np.power(np.linalg.norm(q[0:3]-q[6:9]),3)))
        p_dot[3:6] = -( G*m1*m0*((q[3:6]-q[0:3])/np.power(np.linalg.norm(q[3:6]-q[0:3]),3))
                       +G*m1*m2*((q[3:6]-q[6:9])/np.power(np.linalg.norm(q[3:6]-q[6:9]),3)))
        p_dot[6:9] = -( G*m2*m0*((q[6:9]-q[0:3])/np.power(np.linalg.norm(q[6:9]-q[0:3]),3)) 
                       +G*m2*m1*((q[6:9]-q[3:6])/np.power(np.linalg.norm(q[6:9]-q[3:6]),3)))
        
        return np.concatenate((q_dot, p_dot))
    
    def hamiltonian(self, y):
    
        m0 = self.m0
        m1 = self.m1
        m2 = self.m2
        G = self.G
        nt = y.shape[1]
        neq = y.shape[0]
        q = y[0:neq//2]
        p = y[neq//2:neq]
        ham = np.zeros(nt)

        for i in range(nt):
            ham[i] = ( 0.5*( (1/m0)*np.dot(p[0:3,i],p[0:3,i]) 
                            +(1/m1)*np.dot(p[3:6,i],p[3:6,i]) 
                            +(1/m2)*np.dot(p[6:9,i],p[6:9,i]) ) 
                      - G *( (m0*m1)/np.linalg.norm(q[0:3,i]-q[3:6,i]) 
                            +(m0*m2)/np.linalg.norm(q[0:3,i]-q[6:9,i])
                            +(m1*m2)/np.linalg.norm(q[3:6,i]-q[6:9,i]) ) )

        return ham

In [ ]:
# initialization
m1 = 9.54786104043e-4
m2 = 2.85583733151e-4

qini = np.zeros(9)
qini[0] =  0.;        qini[1] =  0.;        qini[2] =  0.
qini[3] = -3.5023653; qini[4] = -3.8169847; qini[5] = -1.5507963
qini[6] =  9.0755314; qini[7] = -3.0458353; qini[8] = -1.6483708

pini = np.zeros(9)
pini[0] =  0.;            pini[1] =  0.;            pini[2] =  0.
pini[3] =  0.00565429*m1; pini[4] = -0.00412490*m1; pini[5] = -0.00190589*m1
pini[6] =  0.00168318*m2; pini[7] =  0.00483525*m2; pini[8] =  0.00192462*m2

yini = np.concatenate((qini, pini))

tini = 0.
tend = 11000.
nt = 1501

tbm = three_body_model()
fcn = tbm.fcn

sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12)
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=[0], y=[0.], z=[0.], mode="markers", marker=dict(size=8, color="orange"), 
                           showlegend=False))
fig.add_trace(go.Scatter3d(x=[qini[3]], y=[qini[4]], z=[qini[5]], mode="markers", 
                           marker=dict(size=6, color="#EF553B"), showlegend=False))
fig.add_trace(go.Scatter3d(x=sol.y[3]-sol.y[0], y=sol.y[4]-sol.y[0], z=sol.y[5]-sol.y[0], mode='lines', 
                           line_color="#EF553B", line_width=6, name='Jupiter'))
fig.add_trace(go.Scatter3d(x=[qini[6]], y=[qini[7]], z=[qini[8]], mode="markers", 
                           marker=dict(size=6, color="#00CC96"), showlegend=False))
fig.add_trace(go.Scatter3d(x=sol.y[6]-sol.y[0], y=sol.y[7]-sol.y[0], z=sol.y[8]-sol.y[0], mode='lines', 
                           line_color="#00CC96", line_width=6, name='Saturne'))
fig.show()

### Hamiltonien

In [ ]:
ham = tbm.hamiltonian(sol.y)

fig = go.Figure()
fig.add_trace(go.Scatter(x=sol.t, y=ham))
fig.update_layout(title="Hamiltonian")
fig.update_yaxes(range=[-3.157e-8, -3.156e-8], exponentformat='e')
fig.show()